In [21]:
import json
import pandas as pd
import cohere
from tqdm import tqdm
from dotenv import load_dotenv
import os
import re

COHERE_API_KEY = os.getenv('COHERE_API_KEY')
co = cohere.Client(COHERE_API_KEY)

In [3]:
with open('../data/raw/2738.json', 'r') as file:
    json_data = json.load(file)

df = pd.DataFrame(json_data)

In [36]:
selected_data = [{'id': item['id'], 'plain_text': item['plain_text']} for item in json_data]
df = pd.DataFrame(selected_data)

In [37]:
df.isnull().sum()

id            0
plain_text    0
dtype: int64

In [38]:
print(df.iloc[0]['plain_text'])

(Slip Opinion)              OCTOBER TERM, 2022                                       1

                                       Syllabus

         NOTE: Where it is feasible, a syllabus (headnote) will be released, as is
       being done in connection with this case, at the time the opinion is issued.
       The syllabus constitutes no part of the opinion of the Court but has been
       prepared by the Reporter of Decisions for the convenience of the reader.
       See United States v. Detroit Timber & Lumber Co., 200 U. S. 321, 337.


SUPREME COURT OF THE UNITED STATES

                                       Syllabus

 BIDEN, PRESIDENT OF THE UNITED STATES, ET AL.
              v. NEBRASKA ET AL.

   CERTIORARI BEFORE JUDGMENT TO THE UNITED STATES
       COURT OF APPEALS FOR THE EIGHTH CIRCUIT

    No. 22–506.      Argued February 28, 2023—Decided June 30, 2023
Title IV of the Higher Education Act of 1965 (Education Act) governs
  federal financial aid mechanisms, including student l

In [14]:
response = co.embed(
  texts=['hello', 'goodbye'],
  model='embed-english-v3.0',
  input_type='classification'
)
print(response)

cohere.Embeddings {
	embeddings: [[0.016296387, -0.008354187, -0.04699707, -0.07104492, 0.00013196468, -0.014892578, -0.018661499, 0.019134521, 0.008476257, 0.04159546, -0.036895752, -0.00048303604, 0.06414795, -0.036346436, 0.045806885, -0.03125, 0.03793335, 0.048583984, 0.0062179565, 0.0071144104, -0.020935059, 0.04196167, -0.039398193, 0.03463745, 0.051879883, 0.030838013, -0.0048103333, -0.00036287308, -0.017944336, -0.039611816, 0.013389587, 0.0044021606, 0.018951416, 0.020767212, -0.0025997162, 0.0904541, -0.0121154785, -0.026184082, 0.012413025, 0.004119873, 0.030654907, -0.030792236, -0.041107178, -0.02368164, -0.043304443, -0.00077438354, -0.017074585, -0.019729614, 0.078125, -0.031585693, 0.020217896, -0.01524353, 0.017471313, -0.0008010864, -0.03717041, 0.011062622, -0.072143555, -0.013175964, 0.01058197, 0.030853271, 0.044799805, 0.0045928955, 0.03253174, 0.047698975, -0.0039024353, -0.01965332, 0.024475098, -0.013755798, 0.018951416, -0.015487671, 0.015594482, 0.0009632110

In [15]:
response = co.tokenize(
  text='tokenize me! :D',
  model='embed-english-v3.0'
)
print(response)

cohere.Tokens {
	tokens: [19204, 4697, 2033, 999, 1024, 1040]
	token_strings: ['token', '##ize', 'me', '!', ':', 'd']
	meta: {'api_version': {'version': '1'}}
}


In [7]:
def tokenize_text(text):
    response = co.tokenize(
        text=text,
        model='embed-english-v3.0'
    )
    return response

In [27]:
regex_pattern = r"(\n([IVXLCDM]+|[A-Z])\n)"

In [28]:
def chunk_text(text, opinion_id, pattern=regex_pattern):
    sections = re.split(pattern, text)
    labeled_chunks = []

    current_label = None
    for i in range(1, len(sections), 3):
        current_label = sections[i].strip()
        chunk = sections[i + 1].strip()
        labeled_chunks.append({
            'opinion_id': opinion_id,
            'label': current_label,
            'chunk': chunk,
            'char_length': len(chunk)
        })

    return labeled_chunks

In [29]:
chunks_list = []
for index, row in df.iterrows():
    opinion_id = row['id']
    text = row['plain_text']
    chunks_list.extend(chunk_text(text, opinion_id))

chunks_df = pd.DataFrame(chunks_list)

In [30]:
chunks_df.sample(5)

ValueError: a must be greater than 0 unless no samples are taken

In [22]:
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    token_ids, token_strings = tokenize_text(row['plain_text'])
    df.at[index, 'token_ids'] = token_ids
    df.at[index, 'token_strings'] = token_strings
    df.at[index, 'token_count'] = len(token_ids)

  0%|          | 0/88 [00:02<?, ?it/s]


CohereAPIError: invalid request: max character length of text must be less than or equal to 65536 - received 167697 characters